### Import library

In [307]:
import cv2 as cv
import cv2
import numpy as np

### Loading data

In [311]:
targetImagePath = 'data/1.jpg'

In [312]:
# Load the target image with the shapes we're trying to match
target = cv.imread(targetImagePath)
target = resizeImage(target, 30)
target_gray = cv.cvtColor(target, cv.COLOR_BGR2GRAY)
blur_target_gray = cv.medianBlur(target_gray, 9)

# cv.imshow('Target', target_gray)
# cv.waitKey()
# cv.destroyWindow('Target')

Declear standard variable

In [249]:
# kích thước 85,6 mm x 53,98 mm

STANDARD_WIDTH = 856
STANDARD_HEIGHT = 539.8
STANDARD_SIZE_RATIO = STANDARD_WIDTH/STANDARD_HEIGHT

### Preprocessing

#### Histogram balance

In [313]:
# creating a Histograms Equalibzation 
# of a image using cv2.equalizeHist() 
img = target_gray
equ = cv2.equalizeHist(img)
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
cl1 = clahe.apply(img)

# show image input vs output 
cv2.imshow('image origin', img)
cv2.imshow('equalizeHist', equ)
  
cv2.waitKey(0) 
cv2.destroyAllWindows() 

In [317]:
img = target.copy()
img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)

# equalize the histogram of the Y channel
img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])

# convert the YUV image back to RGB format
img_output = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)

cv2.imshow('Color input image', img)
cv2.imshow('Histogram equalized', img_output)

cv2.waitKey(0)
cv2.destroyAllWindows() 

In [316]:
#-----Reading the image-----------------------------------------------------
img = target.copy()

#-----Converting image to LAB Color model----------------------------------- 
lab= cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
# cv2.imshow("lab",lab)

#-----Splitting the LAB image to different channels-------------------------
l, a, b = cv2.split(lab)
# cv2.imshow('l_channel', l)
# cv2.imshow('a_channel', a)
# cv2.imshow('b_channel', b)

#-----Applying CLAHE to L-channel-------------------------------------------
clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
cl = clahe.apply(l)
cv2.imshow('CLAHE output', cl)

#-----Merge the CLAHE enhanced L-channel with the a and b channel-----------
limg = cv2.merge((cl,a,b))
cv2.imshow('limg', limg)

#-----Converting image from LAB Color model to RGB model--------------------
final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
cv2.imshow('final', final)

cv2.waitKey(0)
cv2.destroyAllWindows() 

#### Util function

In [250]:
# # height, width, number of channels in image
# height = img.shape[0]
# width = img.shape[1]
# channels = img.shape[2]

def imageResizeByWidth(img, width=856):
    currRatio = img.shape[0]/img.shape[1]
    print(img.shape)
    height = int(width * currRatio)
    dim = (width, height)
    print(dim)
    # resize image
    resized = cv.resize(img, dim, interpolation = cv.INTER_AREA)
    return resized

In [184]:
def resizeImage(img, percent=100):
    scale_percent = percent # percent of original size
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    # resize image
    resized = cv.resize(img, dim, interpolation = cv.INTER_AREA)
    return resized

In [185]:
def sharpImg(img):
    if len(img.shape)==3:
        img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    else:
        img_gray = img

    kernel_sharpening = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    # applying different kernels to the input image
    return cv.filter2D(img_gray, -1, kernel_sharpening)

#### Get green color from image

In [296]:
frame = target.copy()
# Convert BGR to HSV
hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
# cv.imshow("hsv", hsv)
# define range of blue color in HSV
lower = np.array([40,0,0])
upper = np.array([85,255,255])
# Threshold the HSV image to get only blue colors
mask = cv.inRange(hsv, lower, upper)
# Bitwise-AND mask and original image
res = cv.bitwise_and(frame,frame, mask= mask)
# cv.imshow('frame',frame)
# cv.imshow('mask',mask)
# cv.imshow('res',res)
# cv.waitKey(0) 
# cv.destroyAllWindows()

### Get edge image

In [297]:
# convert to bgr and sharpen image
green_img = cv.cvtColor(res, cv.COLOR_HSV2BGR)
# green_img = cv.cvtColor(green_img, cv.COLOR_BGR2GRAY)
green_sharped = sharpImg(green_img)

In [298]:
kernel = np.ones((7,7), np.uint8)
widerImg = cv.dilate(green_sharped, kernel, iterations = 4)
# cv.imshow('Dilation', dilation)
# cv.waitKey(0)
# cv.destroyAllWindows()

In [299]:
green_edges = cv.Canny(widerImg, 50, 120)
# cv.imshow('green_edges', green_edges)
# cv.waitKey(0)
# cv.destroyWindow('green_edges')

### Find the rectangel bounding the card

In [300]:
sharpened = green_edges
# ret, thresh = cv.threshold(sharpened, 127, 255, 1)

# Extract Contours
# contours, hierarchy = cv.findContours(thresh.copy(), cv.RETR_EXTERNAL,
#                                       cv.CHAIN_APPROX_NONE)

contours, hierarchy = cv.findContours(green_edges, cv.RETR_EXTERNAL,
                                      cv.CHAIN_APPROX_SIMPLE)

contours = sorted(contours, key=cv.contourArea, reverse=True)

In [301]:
# Display contours

# imgShow = target.copy()
# for c in contours[:10]:
#     cv.drawContours(imgShow, [c], -1, (0, 40, 120), 2)
#     cv.imshow('Template_Contours', imgShow)
#     cv.waitKey(1)

# cv.waitKey(0)
# cv.destroyAllWindows()

In [302]:
img = target.copy()
x = y = w = h = None
for cnt in contours:
    # Get approximate polygons
    approx = cv.approxPolyDP(cnt, 0.02 * cv.arcLength(cnt, True), True)
    if len(approx) == 4:
        x, y, w, h = cv.boundingRect(cnt)
        M = cv.moments(cnt)
        print(M)
        if M['m00'] == 0: 
            continue
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])

        # Find contour center to place text at the center
#         shape_name = "Rectangle"
#         cv.drawContours(img, [cnt], 0, (0, 0, 255), 2)
#         cv.putText(img, shape_name, (cx - 50, cy), cv.FONT_HERSHEY_SIMPLEX, 1,
#                    (0, 0, 0), 1)
        break

cv.imshow('Identifying Shapes', img)
cv.waitKey(0)
cv.destroyAllWindows()

{'m00': 818708.0, 'm10': 479048201.5, 'm01': 368253412.6666666, 'm20': 365742396528.5, 'm11': 215132096366.16666, 'm02': 202138012643.66666, 'm30': 313504708319654.94, 'm21': 164055083863421.03, 'm12': 118315537780889.64, 'm03': 123781018427746.2, 'mu20': 85438332856.97784, 'mu11': -342938730.84469604, 'mu02': 36498277914.50037, 'mu30': -485623590946.8125, 'mu21': -53874781216.17969, 'mu12': 347624073200.8203, 'mu03': 25975381103.90625, 'nu20': 0.12746609972007916, 'nu11': -0.0005116329053014442, 'nu02': 0.054452058890810964, 'nu30': -0.0008007135737094176, 'nu21': -8.883066927682458e-05, 'nu12': 0.0005731750251616973, 'nu03': 4.282913890493188e-05}


In [303]:
# cv.imshow('card1', target)
# cv.imshow('card2', target[y:y+h,x:x+w])
# cv.waitKey(0)
# cv.destroyAllWindows()

### Saving the cropped image

In [304]:
croppedImg = target[y:y+h,x:x+w]
print(croppedImg.shape)

(749, 1135, 3)


In [305]:
customSizeImg = imageResizeByWidth(croppedImg, width=856)
# cv.imshow('customSizeImg', customSizeImg)
# cv.waitKey(0)
# cv.destroyAllWindows()

(749, 1135, 3)
(856, 564)


In [306]:
# Filename 
filename = './data/croppedImg2.jpg'
# Saving the image
cv.imwrite(filename, customSizeImg) 

True

### Final Func